In [1]:
import pandas as pd
import os

In [2]:
import json

In [3]:
# check number of files in adult 
DATANAME='heloc'
METHOD = 'ctabsyn'
path = f'ORD/data/{DATANAME}/{METHOD}'


# find all results.csv in all {path}/run*/evaluate/noord and ord and syn_acc.json 


In [5]:
# df = master_df
def aggregate(df):
    df_noord = df[df['cond'] == 'noord']
    df_ord = df[df['cond'] == 'ord']
    df_noord=df_noord.drop(['cond','run','dataset'], axis=1)
    df_ord=df_ord.drop(['cond','run','dataset'], axis=1)
    # Step 2: Calculate mean and std for 'noord'
    mean_noord = df_noord.mean()
    std_noord = df_noord.std()

    # Step 2: Calculate mean and std for 'ord'
    mean_ord = df_ord.mean()
    std_ord = df_ord.std()

    count_noord = len(df_noord)
    count_ord = len(df_ord)
    # Step 3: Create a new DataFrame with these statistics
    summary_df = pd.DataFrame({
        'noord_mean': mean_noord,
        'ord_mean': mean_ord,
        # 'noord_std': std_noord,
        # 'ord_std': std_ord
    })

    # summary_df.loc['count'] = [count_noord, count_ord, count_noord, count_ord]
    summary_df.loc['count'] = [count_noord, count_ord]


    summary = summary_df.T
    return summary

In [6]:
def find_and_aggregate_results(base_path, dataname):
    # Initialize an empty list to store DataFrames
    aggregated_data = []
    aggregated_syn_acc_data = []

    # Traverse the base directory
    for run_dir in os.listdir(base_path):
        run_path = os.path.join(base_path, run_dir)
        if os.path.isdir(run_path):
            # Look for noord and ord directories
            for cond in ['noord', 'ord']:
                results_path = os.path.join(run_path, 'evaluate', cond, 'results.csv')
                if os.path.exists(results_path):
                    # Read the CSV into a DataFrame
                    df = pd.read_csv(results_path)
                    # Add the cond and run columns
                    df['cond'] = cond
                    df['run'] = run_dir
                    df['dataset']=dataname
                    # Append the DataFrame to the list
                    aggregated_data.append(df)
                    
            syn_acc_path = os.path.join(run_path, 'evaluate', 'syn_acc.json')
            if os.path.exists(syn_acc_path):
                with open(syn_acc_path, 'r') as f:
                    syn_acc_data = json.load(f)
                    # Convert the JSON data into a DataFrame with one row
                    syn_acc_df = pd.DataFrame([syn_acc_data])
                    # Add the run column
                    syn_acc_df['run'] = run_dir
                    syn_acc_df['dataset']=dataname
                    # Append the DataFrame to the list
                    aggregated_syn_acc_data.append(syn_acc_df)

     

    # Concatenate all DataFrames into one for results.csv
    if aggregated_data:
        master_results_df = pd.concat(aggregated_data, ignore_index=True)
        # Sort the DataFrame by cond (noord first, then ord)
        master_results_df = master_results_df.sort_values(by='cond', ascending=True).reset_index(drop=True)
    else:
        master_results_df = pd.DataFrame()  # Return an empty DataFrame if no files are found

    # Concatenate all DataFrames into one for syn_acc.json
    if aggregated_syn_acc_data:
        master_syn_acc_df = pd.concat(aggregated_syn_acc_data, ignore_index=True)
    else:
        master_syn_acc_df = pd.DataFrame()  # Return an empty DataFrame if no files are found

    master_results_df = aggregate(master_results_df)
    
    return master_results_df, master_syn_acc_df

# Example usage:
base_path = f'../data/{DATANAME}/{METHOD}'
master_df, syn_df = find_and_aggregate_results(base_path, DATANAME)

# Display the first few rows of the resulting DataFrame
master_df

,XG accuracy,Min accuracy,Maj accuracy,ECE,AUC,ada,dt,lr,mlp,AvgOf4 Acc,count
noord_mean,0.67040,0.58995,0.75085,0.171237,0.733200,0.677579,0.613614,0.679820,0.680734,0.662937,10.0
ord_mean,0.67775,0.59865,0.75685,0.157324,0.736818,0.678080,0.618190,0.677625,0.683988,0.664471,10.0


In [7]:
datas = ['adult', 'cardio' ,'fintech', 'heloc']
methods = ['ctabsyn', 'tabsyn']

main = pd.DataFrame()
syn_main = pd.DataFrame()

for METHOD in methods:
    for DATANAME in datas:
        base_path = f'../data/{DATANAME}/{METHOD}'
        master_df, syn_df = find_and_aggregate_results(base_path, DATANAME)
        master_df['dataset'] = DATANAME
        master_df['method'] = METHOD
        
        syn_df['method'] = METHOD
        
        syn_main = pd.concat([syn_main, syn_df])
        main = pd.concat([main, master_df])

        

In [8]:
main

,XG accuracy,Min accuracy,Maj accuracy,ECE,AUC,ada,dt,lr,mlp,AvgOf4 Acc,count,dataset,method
noord_mean,0.805219,0.824313,0.786125,0.032886,0.890740,0.824536,0.795540,0.795729,0.816336,0.808035,8.0,adult,ctabsyn
ord_mean,0.810219,0.824625,0.795813,0.039350,0.895961,0.822906,0.802707,0.789903,0.816892,0.808102,8.0,adult,ctabsyn
noord_mean,0.723417,0.695944,0.750889,0.033703,0.784837,0.711372,0.686348,0.656413,0.723568,0.694425,9.0,cardio,ctabsyn
ord_mean,0.725375,0.691219,0.759531,0.037134,0.787561,0.710097,0.694012,0.711621,0.721703,0.709358,16.0,cardio,ctabsyn
noord_mean,0.604050,0.528050,0.680050,0.138693,0.649449,0.580335,0.580471,0.597423,0.595281,0.588377,10.0,fintech,ctabsyn
ord_mean,0.663050,0.619050,0.707050,0.109031,0.721054,0.662516,0.613308,0.663328,0.663159,0.650578,10.0,fintech,ctabsyn
noord_mean,0.670400,0.589950,0.750850,0.171237,0.733200,0.677579,0.613614,0.679820,0.680734,0.662937,10.0,heloc,ctabsyn
ord_mean,0.677750,0.598650,0.756850,0.157324,0.736818,0.678080,0.618190,0.677625,0.683988,0.664471,10.0,heloc,ctabsyn
noord_mean,0.797300,0.853700,0.740900,0.017321,0.885570,0.821101,0.792724,0.760179,0.819148,0.798288,5.0,adult,tabsyn
ord_mean,0.809000,0.834000,0.784000,0.033021,0.895044,0.821953,0.803186,0.808181,0.823547,0.814217,5.0,adult,tabsyn


In [9]:
syn_main.drop(['run'],axis=1, inplace=True)
grouped_df = syn_main.groupby(['method', 'dataset'], as_index=False).mean()
grouped_df

,method,dataset,C0,C1,C00,C01,C1(cond)
0,ctabsyn,adult,93.081481,48.567500,94.187315,NaN,51.541667
1,ctabsyn,cardio,76.822083,61.602083,77.856250,15.852075,65.561250
2,ctabsyn,fintech,61.581429,55.191429,78.697143,23.813354,50.572857
3,ctabsyn,heloc,80.522500,53.672500,79.592500,30.194903,54.912500
4,tabsyn,adult,93.119444,42.246667,94.209444,51.038140,49.062222
5,tabsyn,cardio,76.781111,62.762222,77.505556,18.474449,64.416667
6,tabsyn,fintech,77.263095,47.585119,76.700595,29.356876,47.894048
7,tabsyn,heloc,80.882500,48.697500,78.670000,33.231935,49.327500


In [10]:
syn_main

,C0,C1,C00,C01,C1(cond),dataset,method
0,92.722963,48.106667,94.202963,NaN,51.586667,adult,ctabsyn
1,93.440000,49.028333,94.171667,NaN,51.496667,adult,ctabsyn
0,76.850833,65.490833,77.852500,15.846242,65.609167,cardio,ctabsyn
1,76.793333,57.713333,77.860000,15.857907,65.513333,cardio,ctabsyn
0,78.660000,50.882857,78.637143,24.437636,50.294286,fintech,ctabsyn
1,44.502857,59.500000,78.757143,23.189072,50.851429,fintech,ctabsyn
0,80.500000,52.780000,79.590000,29.700150,54.620000,heloc,ctabsyn
1,80.545000,54.565000,79.595000,30.689655,55.205000,heloc,ctabsyn
0,92.988889,38.053333,94.082222,50.697074,49.804444,adult,tabsyn
1,93.250000,46.440000,94.336667,51.379207,48.320000,adult,tabsyn
